In [2]:
# Special configuration for Jupyter Notebook
import nest_asyncio
nest_asyncio.apply() 

In [135]:
import re
import json
import openai

from typing import Dict
from pathlib import Path
from instructor import patch
from pydantic import BaseModel, Field
from pydantic.json import pydantic_encoder
from concurrent.futures import ThreadPoolExecutor

from langchain.llms import OpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

patch()

In [8]:
story_name = 'one-day-prisoner'
story_type = 'txt'
story_root = './stories'
result_root = './results'
instruction_root = './prompts/instructions'
model_name = 'gpt-3.5-turbo-16k'
language = 'Chinese'
chunk_size = 1500
chunk_overlap = 0

In [75]:
def load_text(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def save_text(text, file_path):
    with open(file_path, 'w') as file:
        file.write(text + '\n')

def format_scene(storyboard_result):
    """
    This is a temporary sol and would
    be replace by Pydantic modules later.
    
    storyboard_result (dict): a chain map reduce dict.
    """
    # Initialize a string
    text = ''
    
    # Format the divide line
    for i in storyboard_result['intermediate_steps']:
        text += '\n---\n\n' + i + '\n'
    
    # Format the number which was discarded in map reduce
    scenes = text.split('[Scene]')
    text = scenes[0]
    for j, scene in enumerate(scenes[1:], start=1):
        text += f'[Scene {j}]{scene}'
    
    return text

In [10]:
story_root = Path(story_root)
result_root = Path(result_root)
instruction_root = Path(instruction_root)

In [13]:
# --------------------
# Load the story
# --------------------
# Configure the path
story_path = story_root / f'{story_name}.{story_type}'

# Load with text loader
loader = TextLoader(story_path)
doc = loader.load()

# Split the story into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                               chunk_overlap=chunk_overlap)
split_docs = text_splitter.split_documents(doc)

# --------------------
# Load the prompts
# --------------------
# Load the summary prompt
summary_question_prompt = load_text(instruction_root / 'summary_question.txt')
summary_refine_prompt = load_text(instruction_root / 'summary_refine.txt')

# Load the storyboard prompt
storyboard_map_prompt = load_text(instruction_root / 'storyboard_map.txt')

In [15]:
# ------------------------------------------------
# Get the summary (on characters and environments)
# ------------------------------------------------
# Set the chat model
chat_model = ChatOpenAI(model_name=model_name)

# Set the prompts
question_prompt = PromptTemplate.from_template(summary_question_prompt)
refine_prompt = PromptTemplate.from_template(summary_refine_prompt)

# Run the chain
summary_chain = load_summarize_chain(
    llm=chat_model,
    chain_type='refine',
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key='input_documents',
    output_key='output_text',
)

# Get the results
print('Getting the summary for characters and environments.')
summary_result = summary_chain({'input_documents': split_docs})
summary = summary_result['output_text']

In [22]:
# --------------------
# Get the storyboard
# --------------------
# Set the prompts
map_prompt = PromptTemplate.from_template(storyboard_map_prompt)
combine_prompt = PromptTemplate.from_template('Provide an overall summary of {text}.')

# Run the chain
storyboard_chain = load_summarize_chain(
    llm=chat_model,
    chain_type='map_reduce',
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    input_key='input_documents'
)

# Get the results
print('Getting the storyboard script.')
storyboard_result = storyboard_chain(
    {'input_documents': split_docs,
     'summary': summary})
storyboard = format_scene(storyboard_result)

In [157]:
# --------------------
# Save results
# --------------------
# Configure paths
summary_save_path = result_root / f'{story_name}-summary.txt'
storyboard_save_path = result_root / f'{story_name}-storyboard.txt'

# Save them
save_text(summary, summary_save_path)
save_text(storyboard, storyboard_save_path)

In [159]:
# --------------------
# Translate the content
# --------------------
if language:
    print('Translating the content.')
    
    # Translate for SUMMARY
    summary_translated = chat_model.predict(
        f'Translate the following text in {language}: {summary}')

    # Translate for the STORYBOARD
    # Below is a bit nasty
    # We will update that with Pydantic to keep clean
    def extract_all_scenes(text):
        # Split the text by '---' to get the scenes
        scenes = text.split('---')

        # Remove empty strings and strip leading/trailing white spaces from each scene
        scenes = [scene.strip() for scene in scenes if scene.strip()]

        return scenes

    # Extract all scenes using the revised function
    scenes = extract_all_scenes(storyboard)
    
    # The helper function for scene translation
    def translate_scene(scene):
        translation_prompt = (f'Translate the following text in {language}: \n'
                              f'{scene}\n\n'
                              'YOUR RESPONSE GOES HERE: \n')
        scene_translated = chat_model.predict(translation_prompt) 
        return scene_translated
    
    # List to hold translated scenes
    storyboard_translated_list = []
    
    # Using ThreadPoolExecutor to translate scenes in parallel
    with ThreadPoolExecutor() as executor:
        storyboard_translated_list = list(executor.map(translate_scene, scenes))
        
    # Initialize a string
    storyboard_translated = ''
    
    # Format the divide line
    for i in storyboard_translated_list:
        storyboard_translated += '\n---\n\n' + i + '\n'

    # Save them
    save_text(summary_translated, 
              result_root / f'{story_name}-summary-translated.txt')
    save_text(storyboard_translated, 
              result_root / f'{story_name}-storyboard-translated.txt')

Translating the content.


In [163]:
print(summary)

[Story Summary]
In this mysterious and haunting story, a troubled individual named Mr. B moves into a new building and requests to rent a small, unfurnished room for just one day. Less than 24 hours after his arrival, Mr. B is found dead in his room. Two men in black and a doctor arrive promptly, seemingly aware of his death. They search the room, pack everything into suitcases, and leave with Mr. B's body. The building employee, who narrates the story, reflects on his brief interaction with Mr. B and the strange circumstances surrounding his death. Throughout the day, the narrator witnesses peculiar behavior from Mr. B, raising suspicions about his true identity and motives. Eventually, the narrator discovers a hidden message left by Mr. B on the wall of his room, revealing that he is trapped in an eternal loop, reliving the same day as punishment for a crime. The story ends with the narrator reflecting on the endless nightmare that Mr. B is condemned to.

[Character Summary]
characte

In [164]:
print(storyboard)


---

[Scene 1]
- [Scene Name]: Mr. B's Arrival
- [Environment]: The Building, Nighttime, covered in fog
- [Characters Present]: Mr. B, Two Men in Black, Narrator (Building Employee)
- [Dialogue]: None in this scene
- [Camera Info]: Wide shot of the building exterior, obscured by fog
- [Camera Motion]: Steady shot, slowly moving towards the building entrance
- [Visual Design]: The building is shrouded in fog, creating a mysterious and eerie atmosphere. Mr. B, accompanied by two men in black, stands outside the building entrance. The men carry suitcases. Mr. B appears pale and frail, with a look of sorrow on his face. The narrator, dressed in a uniform, stands at the entrance, observing the scene.
- [Sound Elements]: Eerie ambient music, muffled sounds due to the fog
- [Emotional Atmosphere]: Suspenseful, mysterious

[Text-to-Image Prompt]:
The camera pans across a dark and foggy scene, revealing the large building in the distance. The building's exterior is barely visible, covered in a